In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

from glob import glob
import math

# Import matplotlib
import matplotlib.pyplot as plt
from matplotlib.image import imread
import plotly.express as px

import keras

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

import os # used for navigating to image path
import imageio # used for writing images

import datetime

from PIL import Image
from PIL import ImageFile

##not sure why I need this but I do
ImageFile.LOAD_TRUNCATED_IMAGES = True

pd.set_option('max_colwidth', 400)


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
# load the model
vgg16_model = VGG16(input_shape = (400, 400, 1))

ValueError: When setting `include_top=True` and loading `imagenet` weights, `input_shape` should be (224, 224, 3).

In [2]:
def vgg16_predict(img_path):
    # load an image from file
    image = load_img(img_path, target_size=(224, 224))

    # convert the image pixels to a numpy array
    image = img_to_array(image)
    
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

    # prepare the image for the VGG model
    image = preprocess_input(image)

    # predict the probability across all output classes
    yhat = vgg16_model.predict(image)

    # convert the probabilities to class labels and return the top label
    label = decode_predictions(yhat)[0][0][1]
    
    return label

In [17]:
file_paths = []
image_ids = []
breed_labels = []

for directory in os.listdir('/Users/ilya/Desktop/Computer-Science/U-Capstone/Images'):
    if directory != '.DS_Store':
        for file in os.listdir('/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/'+str(directory)):
            file_paths.append('/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/'
                              +str(directory)+'/'+str(file))
            breed_labels.append(str(directory).split('-')[1])
            image_ids.append(file)

df = pd.DataFrame()

df['file_path'] = file_paths
df['image_id'] = image_ids
df['breed_label'] = breed_labels

df

,file_path,image_id,breed_label
0,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02097658-silky_terrier/n02097658_26.jpg,n02097658_26.jpg,silky_terrier
1,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02097658-silky_terrier/n02097658_4869.jpg,n02097658_4869.jpg,silky_terrier
2,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02097658-silky_terrier/n02097658_595.jpg,n02097658_595.jpg,silky_terrier
3,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02097658-silky_terrier/n02097658_9222.jpg,n02097658_9222.jpg,silky_terrier
4,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02097658-silky_terrier/n02097658_422.jpg,n02097658_422.jpg,silky_terrier
...,...,...,...
20575,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02089078-black-and-tan_coonhound/n02089078_237.jpg,n02089078_237.jpg,black
20576,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02089078-black-and-tan_coonhound/n02089078_222.jpg,n02089078_222.jpg,black
20577,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02089078-black-and-tan_coonhound/n02089078_1021.jpg,n02089078_1021.jpg,black
20578,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02089078-black-and-tan_coonhound/n02089078_183.jpg,n02089078_183.jpg,black


In [18]:
#we count the number of dog images per breed discovering the fact the the dataset in slightly imbalanced

breed_count_df = df.groupby('breed_label').count().reset_index()\
                .rename(columns = {'file_path':'dog_count'})[['breed_label','dog_count']]\
                .sort_values(by = 'dog_count', ascending = False)

df2 = df.merge(breed_count_df, left_on= 'breed_label', right_on = 'breed_label')
df3 = df2.sample(frac = 1)
df4 = df3[df3.dog_count > 160]
df5 = df4[['file_path','image_id','breed_label']]
df5

,file_path,image_id,breed_label
9940,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02097474-Tibetan_terrier/n02097474_1990.jpg,n02097474_1990.jpg,Tibetan_terrier
15281,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02098413-Lhasa/n02098413_5806.jpg,n02098413_5806.jpg,Lhasa
3246,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02110063-malamute/n02110063_19390.jpg,n02110063_19390.jpg,malamute
5716,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02093754-Border_terrier/n02093754_687.jpg,n02093754_687.jpg,Border_terrier
19559,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02112018-Pomeranian/n02112018_888.jpg,n02112018_888.jpg,Pomeranian
...,...,...,...
16231,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02088466-bloodhound/n02088466_9533.jpg,n02088466_9533.jpg,bloodhound
9524,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02107683-Bernese_mountain_dog/n02107683_2059.jpg,n02107683_2059.jpg,Bernese_mountain_dog
7114,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02095570-Lakeland_terrier/n02095570_5068.jpg,n02095570_5068.jpg,Lakeland_terrier
12485,/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02091134-whippet/n02091134_18403.jpg,n02091134_18403.jpg,whippet


In [19]:
train_df = df5.iloc[0:round(len(df5)*0.8)]
test_df = df5.iloc[round(len(df5)*0.8):]

In [20]:
vgg16_pred = vgg16_predict('/Users/ilya/Desktop/Computer-Science/U-Capstone/Images/n02086910-papillon/n02086910_695.jpg')
vgg16_pred

'papillon'

In [21]:
pred_label_list = []

for file_path in test_df['file_path']:
    pred_label = vgg16_predict(file_path)
    pred_label_list.append(pred_label)

In [23]:
test_df.head(100)['pred_label'] = pred_label_list

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
test_df.to_csv('/Users/ilya/Desktop/U-Capstone/data/test_df.csv')

In [39]:
def img_to_np_array(path):
    image = Image.open(path)
    image = image.resize((400,400))
    image = image.convert('L')
    image = np.array(image)
    return image

In [40]:
train_data = []

for file_path, label in zip(train_df['file_path'], train_df['breed_label']):
    img_np_array = img_to_np_array(file_path)
    train_data.append([img_np_array, label])

train_images = np.array([i[0] for i in train_data])
train_labels = np.array([i[1] for i in train_data])

train_images = train_images.reshape(-1, 400, 400, 1)
train_labels = pd.get_dummies(train_labels).to_numpy()

In [42]:
test_data = []

for file_path, label in zip(test_df['file_path'], test_df['breed_label']):
    img_np_array = img_to_np_array(file_path)
    test_data.append([img_np_array, label])

test_images = np.array([i[0] for i in test_data])
test_labels = np.array([i[1] for i in test_data])

test_labels_df = pd.get_dummies(test_labels)

test_images = test_images.reshape(-1, 400, 400, 1)
test_labels = pd.get_dummies(test_labels).to_numpy()

In [36]:
test_labels_df.to_csv(os.path.join(directory,'test_label_column_names.csv'))
print("Test label names saved!")

Test label names saved!


In [43]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(8981, 400, 400, 1)
(8981, 59)
(2245, 400, 400, 1)
(2245, 59)


In [30]:
directory = '/Users/ilya/Desktop/U-Capstone/data'

np.save(os.path.join(directory,'train_images.npy'), train_images)
print("Train images saved!")

np.save(os.path.join(directory,'train_labels.npy'), train_labels)
print("Train labels saved!")

np.save(os.path.join(directory,'test_images.npy'), test_images)
print("Test images saved!")

np.save(os.path.join(directory,'test_labels.npy'), test_labels)
print("Test labels saved!")

Train images saved!
Train labels saved!
Test images saved!
Test labels saved!


## PART III

In [37]:
label_df = pd.read_csv('/Users/ilya/Downloads/label_df.csv').drop(columns = ['Unnamed: 0'])
label_df

,file_path,image_id,breed_label,pred_labels
0,/Users/ilya/Desktop/U-Capstone/Images/n0211127...,n02111277_1193.jpg,Newfoundland,chow
1,/Users/ilya/Desktop/U-Capstone/Images/n0209072...,n02090721_6279.jpg,Irish_wolfhound,Scottish_deerhound
2,/Users/ilya/Desktop/U-Capstone/Images/n0209200...,n02092002_6907.jpg,Scottish_deerhound,Afghan_hound
3,/Users/ilya/Desktop/U-Capstone/Images/n0209200...,n02092002_15342.jpg,Scottish_deerhound,miniature_pinscher
4,/Users/ilya/Desktop/U-Capstone/Images/n0211112...,n02111129_547.jpg,Leonberg,Saluki
...,...,...,...,...
2240,/Users/ilya/Desktop/U-Capstone/Images/n0209146...,n02091467_3767.jpg,Norwegian_elkhound,Scottish_deerhound
2241,/Users/ilya/Desktop/U-Capstone/Images/n0209643...,n02096437_6.jpg,Dandie_Dinmont,Airedale
2242,/Users/ilya/Desktop/U-Capstone/Images/n0209364...,n02093647_1022.jpg,Bedlington_terrier,Scottish_deerhound
2243,/Users/ilya/Desktop/U-Capstone/Images/n0209617...,n02096177_4520.jpg,cairn,Pomeranian


In [39]:
top_true_labels = label_df[label_df['breed_label'] == label_df['pred_labels']].head(10)
top_true_labels

,file_path,image_id,breed_label,pred_labels
6,/Users/ilya/Desktop/U-Capstone/Images/n0208739...,n02087394_7724.jpg,Rhodesian_ridgeback,Rhodesian_ridgeback
30,/Users/ilya/Desktop/U-Capstone/Images/n0211188...,n02111889_13772.jpg,Samoyed,Samoyed
64,/Users/ilya/Desktop/U-Capstone/Images/n0209643...,n02096437_1521.jpg,Dandie_Dinmont,Dandie_Dinmont
85,/Users/ilya/Desktop/U-Capstone/Images/n0209658...,n02096585_1450.jpg,Boston_bull,Boston_bull
102,/Users/ilya/Desktop/U-Capstone/Images/n0208593...,n02085936_126.jpg,Maltese_dog,Maltese_dog
112,/Users/ilya/Desktop/U-Capstone/Images/n0210731...,n02107312_4979.jpg,miniature_pinscher,miniature_pinscher
141,/Users/ilya/Desktop/U-Capstone/Images/n0209200...,n02092002_14771.jpg,Scottish_deerhound,Scottish_deerhound
144,/Users/ilya/Desktop/U-Capstone/Images/n0211080...,n02110806_518.jpg,basenji,basenji
194,/Users/ilya/Desktop/U-Capstone/Images/n0210800...,n02108000_41.jpg,EntleBucher,EntleBucher
207,/Users/ilya/Desktop/U-Capstone/Images/n0209385...,n02093859_2745.jpg,Kerry_blue_terrier,Kerry_blue_terrier


In [40]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from skimage import io

In [41]:
list_of_labels = ["Pred:"+str(pred_label)+", Act:"+str(breed_label) for breed_label, pred_label in zip(top_true_labels['breed_label'], top_true_labels['pred_labels'])]
tuple_of_labels = tuple(list_of_labels)
tuple_of_labels

('Pred:Rhodesian_ridgeback, Act:Rhodesian_ridgeback',
 'Pred:Samoyed, Act:Samoyed',
 'Pred:Dandie_Dinmont, Act:Dandie_Dinmont',
 'Pred:Boston_bull, Act:Boston_bull',
 'Pred:Maltese_dog, Act:Maltese_dog',
 'Pred:miniature_pinscher, Act:miniature_pinscher',
 'Pred:Scottish_deerhound, Act:Scottish_deerhound',
 'Pred:basenji, Act:basenji',
 'Pred:EntleBucher, Act:EntleBucher',
 'Pred:Kerry_blue_terrier, Act:Kerry_blue_terrier')

In [ ]:
'/Users/ilya/Desktop/U-Capstone/Images/n02087394-Rhodesian_ridgeback/n02087394_381.jpg'

In [45]:
directory = '/Users/ilya/Desktop/U-Capstone/Images'

fig = make_subplots(rows=1, cols=10, subplot_titles=tuple_of_labels)

for label, image_id, i in zip(top_true_labels['breed_label'], top_true_labels['image_id'], range(1, len(top_true_labels)+1)):
    
    path = os.path.join(directory, image_id[0:9]+'-'+label, image_id)
    
    img = io.imread(path)
    
    fig.add_trace(go.Image(z=img), 1, i)

fig.show()


In [48]:
samoyed = img_to_np_array('/Users/ilya/Desktop/russian-bee.png').reshape(1,60,60,1)

ValueError: cannot reshape array of size 40000 into shape (1,60,60,1)

In [46]:
prediction = model.predict(samoyed)

In [57]:
%pprint

Pretty printing has been turned OFF


In [47]:
label_list = list(train_labels_df.columns)
label_list

['Afghan_hound',
 'Airedale',
 'Bernese_mountain_dog',
 'EntleBucher',
 'Great_Pyrenees',
 'Irish_wolfhound',
 'Leonberg',
 'Maltese_dog',
 'Pomeranian',
 'Samoyed',
 'Scottish_deerhound',
 'Sealyham_terrier',
 'Shih',
 'Tibetan_terrier',
 'basenji']

In [48]:
prediction_list = list(prediction[0])
prediction_list

[0.061776843,
 0.06985099,
 0.05059278,
 0.056825694,
 0.05707421,
 0.05678923,
 0.06951023,
 0.08338115,
 0.094307125,
 0.055790253,
 0.070337705,
 0.07566053,
 0.06818018,
 0.06824093,
 0.06168212]

In [49]:
predict_dict = {}

for a, b in zip(label_list, prediction_list):
    predict_dict[a] = b

In [50]:
predict_dict

{'Afghan_hound': 0.061776843,
 'Airedale': 0.06985099,
 'Bernese_mountain_dog': 0.05059278,
 'EntleBucher': 0.056825694,
 'Great_Pyrenees': 0.05707421,
 'Irish_wolfhound': 0.05678923,
 'Leonberg': 0.06951023,
 'Maltese_dog': 0.08338115,
 'Pomeranian': 0.094307125,
 'Samoyed': 0.055790253,
 'Scottish_deerhound': 0.070337705,
 'Sealyham_terrier': 0.07566053,
 'Shih': 0.06818018,
 'Tibetan_terrier': 0.06824093,
 'basenji': 0.06168212}